In [1]:
import pandas as pd
import numpy as np

import  matplotlib.pyplot  as plt
% matplotlib inline
% load_ext autoreload
% autoreload 2

In [141]:
lifeExpPath = 'data/IHME_USA_COUNTY_LE_ONLY_1980_2014_NATIONAL_Y2017M05D08.csv'
censusPath = 'data/cc-est2016-alldata.csv'
leCols = ['Location', 'FIPS', 'Life expectancy, 2010*', 'Life expectancy, 2014*']
lifeExpDF = pd.read_csv(lifeExpPath, dtype = {'FIPS': int}, skiprows = [1, 3195, 3196],
                       usecols = leCols)
censusCols = ["STATE", "COUNTY", "YEAR", "AGEGRP", "TOT_POP", "STNAME", "CTYNAME",# general information.
              "IA_FEMALE", "IA_MALE", "IAC_FEMALE", "IAC_MALE",# American Indian and Alaskan Native only
                                                               # or American Indian and Alaskan Native only+Combo
              "NA_FEMALE", "NA_MALE", "NAC_FEMALE", "NAC_MALE",]# Same as above but with Native Hawaiians and
                                                               # pacific islanders
#               "HIA_FEMALE", "HIA_MALE", "HIAC_FEMALE", "HIAC_MALE", # Same as the first group but with Hispanics
#               "HNA_FEMALE", "HNA_MALE", "HNAC_FEMALE", "HNAC_MALE"] # Same as second group but with Hispanics
censusDF = pd.read_csv(censusPath, encoding = "ISO-8859-1", usecols = censusCols,
                      dtype = {"STATE": str, "COUNTY": str})
censusDF = censusDF[censusDF.AGEGRP == 0]
censusDF = censusDF[censusDF.YEAR.isin([3, 7])]# these years correspond to 2010 and 2014. 2010 is a bit weird
# in that there are 3 estimates for it. As far as I can tell the third one is the most like the 2014 census data.
# this was determined using the census Data methedology pdf.
censusDF.STATE = censusDF.STATE.apply(lambda x: x.lstrip("0"))
censusDF.insert(0, "FIPS", censusDF.STATE + censusDF.COUNTY)
censusDF.drop(["STATE", "COUNTY"], axis = 1, inplace = True)
censusDF.FIPS = censusDF.FIPS.astype(int)

In [142]:
# extract out the census data into year specific dataframes
censusDF_2010 = censusDF[censusDF.YEAR == 3]
censusDF_2014 = censusDF[censusDF.YEAR == 7]

In [143]:
# Change from raw population to percentage of total of county population.
censusDF_2014.iloc[:,6:] = censusDF_2014.iloc[:,6:].divide(censusDF_2014.TOT_POP, axis = 'index')
censusDF_2010.iloc[:,6:] = censusDF_2010.iloc[:,6:].divide(censusDF_2010.TOT_POP, axis = 'index')

/home/ob2285/anaconda3/envs/lifeExpCount/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [144]:
# Remove confidence intervals and convert LE to a float
lifeExpDF['Life expectancy, 2010*'] = lifeExpDF['Life expectancy, 2010*'].apply(lambda x: float(x.split()[0]))
lifeExpDF['Life expectancy, 2014*'] = lifeExpDF['Life expectancy, 2014*'].apply(lambda x: float(x.split()[0]))

In [145]:
lifeExpDF.head()

,Location,FIPS,"Life expectancy, 2010*","Life expectancy, 2014*"
0,Alabama,1,75.58,75.65
1,"Autauga County, Alabama",1001,75.74,75.67
2,"Baldwin County, Alabama",1003,77.80,78.08
3,"Barbour County, Alabama",1005,75.34,75.42
4,"Bibb County, Alabama",1007,74.13,73.97


In [146]:
censusDF.head()

,FIPS,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,IA_MALE,IA_FEMALE,NA_MALE,NA_FEMALE,IAC_MALE,IAC_FEMALE,NAC_MALE,NAC_FEMALE
38,1001,Alabama,Autauga County,3,0,54742,116,135,29,17,258,301,42,33
114,1001,Alabama,Autauga County,7,0,54977,122,153,32,20,286,307,52,47
209,1003,Alabama,Baldwin County,3,0,183199,664,674,76,54,1303,1333,124,117
285,1003,Alabama,Baldwin County,7,0,199745,759,756,63,54,1506,1551,137,145
380,1005,Alabama,Barbour County,3,0,27348,87,72,29,17,150,108,35,19


In [147]:
censusDF_2010.head()

,FIPS,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,IA_MALE,IA_FEMALE,NA_MALE,NA_FEMALE,IAC_MALE,IAC_FEMALE,NAC_MALE,NAC_FEMALE
38,1001,Alabama,Autauga County,3,0,54742,0.002119,0.002466,0.000530,0.000311,0.004713,0.005499,0.000767,0.000603
209,1003,Alabama,Baldwin County,3,0,183199,0.003624,0.003679,0.000415,0.000295,0.007112,0.007276,0.000677,0.000639
380,1005,Alabama,Barbour County,3,0,27348,0.003181,0.002633,0.001060,0.000622,0.005485,0.003949,0.001280,0.000695
551,1007,Alabama,Bibb County,3,0,22861,0.001531,0.001444,0.001181,0.000175,0.003631,0.003456,0.001356,0.000306
722,1009,Alabama,Blount County,3,0,57376,0.003120,0.003067,0.000627,0.000418,0.006205,0.006553,0.000819,0.000645


In [148]:
censusDF_2014.head()

,FIPS,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,IA_MALE,IA_FEMALE,NA_MALE,NA_FEMALE,IAC_MALE,IAC_FEMALE,NAC_MALE,NAC_FEMALE
114,1001,Alabama,Autauga County,7,0,54977,0.002219,0.002783,0.000582,0.000364,0.005202,0.005584,0.000946,0.000855
285,1003,Alabama,Baldwin County,7,0,199745,0.003800,0.003785,0.000315,0.000270,0.007540,0.007765,0.000686,0.000726
456,1005,Alabama,Barbour County,7,0,26763,0.003400,0.002466,0.000971,0.000635,0.005605,0.003774,0.001158,0.000822
627,1007,Alabama,Bibb County,7,0,22511,0.001910,0.001821,0.001066,0.000133,0.003465,0.004042,0.001333,0.000444
798,1009,Alabama,Blount County,7,0,57621,0.003245,0.002898,0.000677,0.000382,0.006369,0.007168,0.000885,0.000694


In [149]:
lifeExpDF.FIPS.head()

0       1
1    1001
2    1003
3    1005
4    1007
Name: FIPS, dtype: int64

In [ ]:
censusDF_2010.FIP

In [160]:
# merge dataframes on FIPS and remove some redundant columns
lifeExpRaceDF_2010 = lifeExpDF.merge(censusDF_2010, how = "inner", on = "FIPS")
lifeExpRaceDF_2014 = lifeExpDF.merge(censusDF_2014, how = "inner", on = "FIPS")
lifeExpRaceDF_2010.drop(labels = ["YEAR", "AGEGRP"], axis = 1, inplace = True)
lifeExpRaceDF_2014.drop(labels = ["YEAR", "AGEGRP"], axis = 1, inplace = True)

In [164]:
lifeExpRaceDF_2010.head(n = 5)

,Location,FIPS,"Life expectancy, 2010*","Life expectancy, 2014*",STNAME,CTYNAME,TOT_POP,IA_MALE,IA_FEMALE,NA_MALE,NA_FEMALE,IAC_MALE,IAC_FEMALE,NAC_MALE,NAC_FEMALE
0,"Autauga County, Alabama",1001,75.74,75.67,Alabama,Autauga County,54742,0.002119,0.002466,0.000530,0.000311,0.004713,0.005499,0.000767,0.000603
1,"Baldwin County, Alabama",1003,77.80,78.08,Alabama,Baldwin County,183199,0.003624,0.003679,0.000415,0.000295,0.007112,0.007276,0.000677,0.000639
2,"Barbour County, Alabama",1005,75.34,75.42,Alabama,Barbour County,27348,0.003181,0.002633,0.001060,0.000622,0.005485,0.003949,0.001280,0.000695
3,"Bibb County, Alabama",1007,74.13,73.97,Alabama,Bibb County,22861,0.001531,0.001444,0.001181,0.000175,0.003631,0.003456,0.001356,0.000306
4,"Blount County, Alabama",1009,76.41,76.16,Alabama,Blount County,57376,0.003120,0.003067,0.000627,0.000418,0.006205,0.006553,0.000819,0.000645


In [165]:
lifeExpRaceDF_2014.head(n = 5)

,Location,FIPS,"Life expectancy, 2010*","Life expectancy, 2014*",STNAME,CTYNAME,TOT_POP,IA_MALE,IA_FEMALE,NA_MALE,NA_FEMALE,IAC_MALE,IAC_FEMALE,NAC_MALE,NAC_FEMALE
0,"Autauga County, Alabama",1001,75.74,75.67,Alabama,Autauga County,54977,0.002219,0.002783,0.000582,0.000364,0.005202,0.005584,0.000946,0.000855
1,"Baldwin County, Alabama",1003,77.80,78.08,Alabama,Baldwin County,199745,0.003800,0.003785,0.000315,0.000270,0.007540,0.007765,0.000686,0.000726
2,"Barbour County, Alabama",1005,75.34,75.42,Alabama,Barbour County,26763,0.003400,0.002466,0.000971,0.000635,0.005605,0.003774,0.001158,0.000822
3,"Bibb County, Alabama",1007,74.13,73.97,Alabama,Bibb County,22511,0.001910,0.001821,0.001066,0.000133,0.003465,0.004042,0.001333,0.000444
4,"Blount County, Alabama",1009,76.41,76.16,Alabama,Blount County,57621,0.003245,0.002898,0.000677,0.000382,0.006369,0.007168,0.000885,0.000694


In [166]:
# Write those puppies to csv my dude
lifeExpRaceDF_2010.to_csv("data/LE_censusData_2010.csv")
lifeExpRaceDF_2010.to_csv("data/LE_censusData_2014.csv")

In [159]:
# this information is also in the censusDataKey pdf but it's helpful to put it here too
yearDict = {"1":"4/1/2010 Census population",
            "2":"4/1/2010 population estimates base",
            "3":"7/1/2010 population estimate",
            "4":"7/1/2011 population estimate",
            "5":"7/1/2012 population estimate",
            "6":"7/1/2013 population estimate",
            "7":"7/1/2014 population estimate",
            "8":"7/1/2015 population estimate",
            "9":"7/1/2016 population estimate",}# Maps year ID to human readable year
ageGrpDict = {"0":"Total",
              "1":"Age 0 to 4 years",
              "2":"Age 5 to 9 years",
              "3":"Age 10 to 14 years",
              "4":"Age 15 to 19 years",
              "5":"Age 20 to 24 years",
              "6":"Age 25 to 29 years",
              "7":"Age 30 to 34 years",
              "8":"Age 35 to 39 years",
              "9":"Age 40 to 44 years",
              "10":"Age 45 to 49 years",
              "11":"Age 50 to 54 years",
              "12":"Age 55 to 59 years",
              "13":"Age 60 to 64 years",
              "14":"Age 65 to 69 years",
              "15":"Age 70 to 74 years",
              "16":"Age 75 to 79 years",
              "17":"Age 80 to 84 years",
              "18":"Age 85 years or older",}# Maps age group ID to year. 

In [19]:
censusDF[censusDF.AGEGRP == 0]

,STATE,COUNTY,YEAR,AGEGRP,TOT_POP,IA_MALE,IA_FEMALE,NA_MALE,NA_FEMALE,IAC_MALE,...,NAC_MALE,NAC_FEMALE,HIA_MALE,HIA_FEMALE,HNA_MALE,HNA_FEMALE,HIAC_MALE,HIAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,1,1,1,0,54571,119,139,29,18,277,...,42,37,16,24,16,9,26,32,19,10
19,1,1,2,0,54571,119,139,29,18,277,...,42,37,16,24,16,9,26,32,19,10
38,1,1,3,0,54742,116,135,29,17,258,...,42,33,9,16,15,7,16,29,17,11
57,1,1,4,0,55255,117,141,34,19,266,...,47,38,8,17,13,6,26,32,15,7
76,1,1,5,0,55027,119,155,29,17,275,...,47,40,9,16,11,6,27,30,20,9
95,1,1,6,0,54792,123,150,30,19,278,...,62,46,12,14,9,5,37,33,13,11
114,1,1,7,0,54977,122,153,32,20,286,...,52,47,10,19,9,5,32,36,12,12
133,1,1,8,0,55035,120,148,29,24,271,...,45,56,14,21,10,6,29,34,14,16
152,1,1,9,0,55416,124,145,29,27,295,...,50,48,20,16,12,8,44,42,15,12
171,1,3,1,0,182265,662,677,72,49,1295,...,127,84,104,85,25,10,189,163,45,26


In [30]:
temp = "1"